In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q evaluate
!pip install -q seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [4]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset

## 训练数据准备

In [5]:
# 加载hf中dataset
ds = load_dataset('nlhappy/CLUE-NER')
ds

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

(…)-00000-of-00001-a33d0e4276aef9b4.parquet:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

(…)-00000-of-00001-07f476b71c5edde6.parquet:   0%|          | 0.00/178k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10748 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1343 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'ents'],
        num_rows: 10748
    })
    validation: Dataset({
        features: ['text', 'ents'],
        num_rows: 1343
    })
})

In [6]:
train_data = ds['train']
print("train_data[0]:", train_data[0])
 
# 初始化一个集合来存储所有唯一的标签
all_labels = set()
 
# 遍历每个样本
for sample in train_data:
    # 检查样本中是否包含 'ents' 字段
    if 'ents' in sample:
        # 遍历每个实体
        for entity in sample['ents']:
            # 提取标签并添加到集合中
            if 'label' in entity:
                all_labels.add(entity['label'])
 
# 打印所有唯一的标签
print("All unique labels:", all_labels)

train_data[0]: {'text': '浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，', 'ents': [{'indices': [9, 10, 11], 'is_continuous': True, 'label': 'name', 'text': '叶老桂'}, {'indices': [0, 1, 2, 3], 'is_continuous': True, 'label': 'company', 'text': '浙商银行'}]}
All unique labels: {'organization', 'company', 'scene', 'book', 'name', 'game', 'movie', 'address', 'position', 'government'}


In [7]:
# entity_index
entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', \
           'company', 'scene', 'book', 'organization', 'government'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())

entity_index = {entity:i for i, entity in enumerate(entites)}

In [8]:
def entity_tags_proc(item):
    # item即是dataset中记录
    text_len = len(item['text'])  # 根据文本长度生成tags列表
    tags = [0] * text_len    # 初始值为‘O’
    # 遍历实体列表，所有实体类别标记填入tags
    entites = item['ents']
    for ent in entites:
        indices = ent['indices']  # 实体索引
        label = ent['label']   # 实体名
        tags[indices[0]] = entity_index[label] * 2 - 1
        for idx in indices[1:]:
            tags[idx] = entity_index[label] * 2
    return {'ent_tag': tags}

# 使用自定义回调函数处理数据集记录
ds1 = ds.map(entity_tags_proc)
ds1

Map:   0%|          | 0/10748 [00:00<?, ? examples/s]

Map:   0%|          | 0/1343 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'ents', 'ent_tag'],
        num_rows: 10748
    })
    validation: Dataset({
        features: ['text', 'ents', 'ent_tag'],
        num_rows: 1343
    })
})

In [9]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [10]:
def data_input_proc(item):
    # 输入文本先拆分为字符，再转换为模型输入的token索引
    batch_texts = [list(text) for text in item['text']]
    # 导入拆分为字符的文本列表时，需要设置参数is_split_into_words=True
    input_data = tokenizer(batch_texts, truncation=True, add_special_tokens=False, max_length=512,
                           is_split_into_words=True, padding='max_length')
    input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ent_tag']]
    return input_data


ds2 = ds1.map(data_input_proc, batched=True)  # batched 每次传入自定义方法样本数量多个

# 记录转换为pytorch
ds2.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

ds2

Map:   0%|          | 0/10748 [00:00<?, ? examples/s]

Map:   0%|          | 0/1343 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'ents', 'ent_tag', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10748
    })
    validation: Dataset({
        features: ['text', 'ents', 'ent_tag', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1343
    })
})

In [11]:
from torch.utils.data import DataLoader
train_dl = DataLoader(ds2['train'], shuffle=True, batch_size=16)

id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

In [12]:
print(id2lbl)

{0: 'O', 1: 'B-ORGANIZATION', 2: 'I-ORGANIZATION', 3: 'B-COMPANY', 4: 'I-COMPANY', 5: 'B-SCENE', 6: 'I-SCENE', 7: 'B-BOOK', 8: 'I-BOOK', 9: 'B-NAME', 10: 'I-NAME', 11: 'B-GAME', 12: 'I-GAME', 13: 'B-MOVIE', 14: 'I-MOVIE', 15: 'B-ADDRESS', 16: 'I-ADDRESS', 17: 'B-POSITION', 18: 'I-POSITION', 19: 'B-GOVERNMENT', 20: 'I-GOVERNMENT'}


In [13]:
print(lbl2id)

{'O': 0, 'B-ORGANIZATION': 1, 'I-ORGANIZATION': 2, 'B-COMPANY': 3, 'I-COMPANY': 4, 'B-SCENE': 5, 'I-SCENE': 6, 'B-BOOK': 7, 'I-BOOK': 8, 'B-NAME': 9, 'I-NAME': 10, 'B-GAME': 11, 'I-GAME': 12, 'B-MOVIE': 13, 'I-MOVIE': 14, 'B-ADDRESS': 15, 'I-ADDRESS': 16, 'B-POSITION': 17, 'I-POSITION': 18, 'B-GOVERNMENT': 19, 'I-GOVERNMENT': 20}


## 模型训练

### 1. 动态学习率

In [23]:
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim
from tqdm import tqdm

DEVICE='cuda'

model1 = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                        num_labels=len(tags),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model1.to(DEVICE)

# 模型参数分组
param_optimizer1 = list(model1.named_parameters())
bert_params1, classifier_params1 = [],[]

for name,params in param_optimizer1:
    if 'bert' in name:
        bert_params1.append(params)
    else:
        classifier_params1.append(params)

param_groups1 = [
    {'params':bert_params1, 'lr':1e-5},
    {'params':classifier_params1, 'weight_decay':0.1, 'lr':1e-3}
]

# optimizer
optimizer1 = optim.AdamW(param_groups1) # 优化器

epochs = 2

# 学习率调度器
train_steps = len(train_dl) * epochs
scheduler1 = get_linear_schedule_with_warmup(optimizer1,
                                            num_warmup_steps=100,
                                            num_training_steps=train_steps)

for epoch in range(epochs):
    model1.train()
    tpbar = tqdm(train_dl)
    for items in tpbar:
        items = {k:v.to(DEVICE) for k,v in items.items()}
        optimizer1.zero_grad()
        outputs = model1(**items)
        loss = outputs.loss
        loss.backward()
        optimizer1.step()
        scheduler1.step()

        tpbar.set_description(f'Epoch:{epoch+1} ' +
                          f'bert_lr:{scheduler1.get_lr()[0]:.8f} ' +
                          f'classifier_lr:{scheduler1.get_lr()[1]:.8f} '+
                          f'Loss:{loss.item():.4f}')

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:1 bert_lr:0.00000540 classifier_lr:0.00054019 Loss:0.0363: 100%|██████████| 672/672 [17:25<00:00,  1.56s/it]
Epoch:2 bert_lr:0.00000000 classifier_lr:0.00000000 Loss:0.0313: 100%|██████████| 672/672 [17:24<00:00,  1.55s/it]


### 2.使用混合精度

In [26]:
import torch

model2 = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                        num_labels=len(tags),
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model2.to(DEVICE)

# 模型参数分组
param_optimizer2 = list(model2.named_parameters())
bert_params2, classifier_params2 = [],[]

for name,params in param_optimizer2:
    if 'bert' in name:
        bert_params2.append(params)
    else:
        classifier_params2.append(params)

param_groups2 = [
    {'params':bert_params2, 'lr':1e-5},
    {'params':classifier_params2, 'weight_decay':0.1, 'lr':1e-3}
]

# optimizer
optimizer2 = optim.AdamW(param_groups2) # 优化器

epochs = 2

# 学习率调度器
train_steps = len(train_dl) * epochs
scheduler2 = get_linear_schedule_with_warmup(optimizer2,
                                            num_warmup_steps=100,
                                            num_training_steps=train_steps)

# 梯度计算缩放器
scaler = torch.GradScaler()

for epoch in range(epochs):
    model2.train()
    tpbar = tqdm(train_dl)
    for items in tpbar:
        items = {k:v.to(DEVICE) for k,v in items.items()}
        optimizer2.zero_grad()

        with torch.autocast(device_type='cuda'):
            outputs = model2(**items)
        loss = outputs.loss

        # 缩放loss后，调用backward
        scaler.scale(loss).backward()
        scaler.step(optimizer2)
        scaler.update()
        scheduler2.step()

        tpbar.set_description(f'Epoch:{epoch+1} ' +
                          f'bert_lr:{scheduler2.get_lr()[0]:.8f} ' +
                          f'classifier_lr:{scheduler2.get_lr()[1]:.8f} '+
                          f'Loss:{loss.item():.4f}')

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:1 bert_lr:0.00000540 classifier_lr:0.00054019 Loss:0.0375: 100%|██████████| 672/672 [04:32<00:00,  2.47it/s]
Epoch:2 bert_lr:0.00000000 classifier_lr:0.00000000 Loss:0.0172: 100%|██████████| 672/672 [04:31<00:00,  2.48it/s]


### 3.使用 DDP 训练

In [21]:
%%writefile ddp_train.py

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from datasets import load_dataset
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

# 设置分布式环境
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

# 清理分布式环境
def cleanup():
    dist.destroy_process_group()

# 定义训练循环
def train(rank, world_size):
    setup(rank, world_size)

    # 数据集
    ds = load_dataset('nlhappy/CLUE-NER')
    # entity_index
    entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', \
               'company', 'scene', 'book', 'organization', 'government'})
    tags = ['O']
    for entity in entites[1:]:
        tags.append('B-' + entity.upper())
        tags.append('I-' + entity.upper())

    entity_index = {entity:i for i, entity in enumerate(entites)}

    tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

    def entity_tags_proc(item):
        # item即是dataset中记录
        text_len = len(item['text'])  # 根据文本长度生成tags列表
        tags = [0] * text_len    # 初始值为‘O’
        # 遍历实体列表，所有实体类别标记填入tags
        entites = item['ents']
        for ent in entites:
            indices = ent['indices']  # 实体索引
            label = ent['label']   # 实体名
            tags[indices[0]] = entity_index[label] * 2 - 1
            for idx in indices[1:]:
                tags[idx] = entity_index[label] * 2
        return {'ent_tag': tags}

    # 使用自定义回调函数处理数据集记录
    ds1 = ds.map(entity_tags_proc)

    def data_input_proc(item):
        # 输入文本先拆分为字符，再转换为模型输入的token索引
        batch_texts = [list(text) for text in item['text']]
        # 导入拆分为字符的文本列表时，需要设置参数is_split_into_words=True
        input_data = tokenizer(
            batch_texts,
            truncation=True,
            add_special_tokens=False,
            max_length=512,
            is_split_into_words=True,
            padding='max_length'
        )

        input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ent_tag']]
        return input_data


    ds2 = ds1.map(data_input_proc, batched=True)  # batch_size 1000

    ds2.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

    # 分布式训练采样器
    sampler = DistributedSampler(ds2['train'], num_replicas=world_size, rank=rank)
    train_dl = DataLoader(ds2['train'], sampler=sampler, batch_size=16)

    id2lbl = {i:tag for i, tag in enumerate(tags)}
    lbl2id = {tag:i for i, tag in enumerate(tags)}

    # 定义模型并将其移动到对应的 GPU 设备端
    model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                            num_labels=len(tags),
                                                            id2label=id2lbl,
                                                            label2id=lbl2id)
    model = model.to(rank)

    ddp_model = DDP(model, device_ids=[rank])

    # 模型参数分组
    param_optimizer = list(model.named_parameters())
    bert_params, classifier_params = [],[]

    for name,params in param_optimizer:
        if 'bert' in name:
            bert_params.append(params)
        else:
            classifier_params.append(params)
    
    param_groups = [
        {'params':bert_params, 'lr':1e-5},
        {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3}
    ]
    
    # optimizer
    optimizer = optim.AdamW(param_groups) # 优化器

    epochs = 2
    
    # 学习率调度器
    train_steps = len(train_dl) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=100,
                                                num_training_steps=train_steps)

    # 在训练开始时创建一次
    scaler = torch.GradScaler()
    # scaler = torch.cuda.amp.GradScaler()

    for epoch in range(epochs):
        ddp_model.train()
        tpbar = tqdm(train_dl)
        for items in tpbar:
            items = {k:v.to(rank) for k,v in items.items()}
            optimizer.zero_grad()
    
            # with torch.autocast(device_type='cuda'):
            with torch.cuda.amp.autocast():
                outputs = ddp_model(**items)
            loss = outputs.loss
    
            # 缩放loss后，调用backward
            scaler.scale(loss).backward()
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
    
            # print(f"Rank {rank}, Epoch {epoch}, Loss: {loss.item()}")
    
            tpbar.set_description(f'Epoch:{epoch+1} ' +
                              f'bert_lr:{scheduler.get_lr()[0]:.8f} ' +
                              f'classifier_lr:{scheduler.get_lr()[1]:.8f} '+
                              f'Loss:{loss.item():.4f}')

    cleanup()

def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)

if __name__ == "__main__":
    main()

Overwriting ddp_train.py


In [22]:
!python ddp_train.py

2025-06-13 06:52:07.243231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749797527.265326     663 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749797527.271913     663 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-13 06:52:16.279207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749797536.301761     677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749797536.308920     677 cuda_blas.cc:1

### 4. Trainer 模型训练整合

In [21]:
%%writefile trainer_full_train.py

import os
import numpy as np
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp

# 设置分布式环境
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

# 清理分布式环境
def cleanup():
    dist.destroy_process_group()

def train(rank, world_size):
    setup(rank, world_size)
    # 数据集
    ds = load_dataset('nlhappy/CLUE-NER')
    # entity_index
    entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', \
               'company', 'scene', 'book', 'organization', 'government'})
    tags = ['O']
    for entity in entites[1:]:
        tags.append('B-' + entity.upper())
        tags.append('I-' + entity.upper())

    entity_index = {entity:i for i, entity in enumerate(entites)}

    tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

    def entity_tags_proc(item):
        # item即是dataset中记录
        text_len = len(item['text'])  # 根据文本长度生成tags列表
        tags = [0] * text_len    # 初始值为‘O’
        # 遍历实体列表，所有实体类别标记填入tags
        entites = item['ents']
        for ent in entites:
            indices = ent['indices']  # 实体索引
            label = ent['label']   # 实体名
            tags[indices[0]] = entity_index[label] * 2 - 1
            for idx in indices[1:]:
                tags[idx] = entity_index[label] * 2
        return {'ent_tag': tags}

    # 使用自定义回调函数处理数据集记录
    ds1 = ds.map(entity_tags_proc)

    def data_input_proc(item):
        # 输入文本先拆分为字符，再转换为模型输入的token索引
        batch_texts = [list(text) for text in item['text']]
        # 导入拆分为字符的文本列表时，需要设置参数is_split_into_words=True
        input_data = tokenizer(batch_texts, truncation=True, add_special_tokens=False, max_length=512,
                               is_split_into_words=True, padding='max_length')
        input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ent_tag']]
        return input_data


    ds2 = ds1.map(data_input_proc, batched=True)  # batch_size 1000

    local_rank = rank

    id2lbl = {i:tag for i, tag in enumerate(tags)}
    lbl2id = {tag:i for i, tag in enumerate(tags)}

    model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                            num_labels=len(tags),
                                                            id2label=id2lbl,
                                                            label2id=lbl2id)
    model.to(local_rank)

    # 冻结模型的所有参数，除了分类层
    # for name, param in model.named_parameters():
    #     if 'classifier' not in name:
    #         param.requires_grad = False

    args = TrainingArguments(
        output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
        num_train_epochs = 2,    # 训练 epoch
        # save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
        per_device_train_batch_size=16, # 训练批次
        per_device_eval_batch_size=16,
        # report_to='tensorboard',  # 训练输出记录
        report_to="none",  # 禁用所有报告
        # save_strategy="steps",  # 减少保存频率
        # save_steps=1000,
        # logging_strategy="steps",  # 减少日志频率
        # logging_steps=1000,
        eval_strategy="epoch",
        local_rank=local_rank,   # 当前进程 RANK
        fp16=True,               # 使用混合精度
        lr_scheduler_type='linear',  # 动态学习率
        warmup_steps=100,        # 预热步数
        # dataloader_num_workers=4,  # 增加数据加载线程数
        ddp_find_unused_parameters=False  # 优化DDP性能
    )

    def compute_metric(result):
        # result 是一个tuple (predicts, labels)

        # 获取评估对象
        seqeval = evaluate.load('seqeval')
        predicts,labels = result
        predicts = np.argmax(predicts, axis=2)

        # 准备评估数据
        predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        results = seqeval.compute(predictions=predicts, references=labels)

        return results

    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

    trainer = Trainer(
        model,
        args,
        train_dataset=ds2['train'],
        eval_dataset=ds2['validation'],
        data_collator=data_collator,
        compute_metrics=compute_metric
    )

    trainer.train()

    torch.save(model.state_dict(), 'model.bin')

def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)

if __name__ == "__main__":
    main()

Overwriting trainer_full_train.py


In [22]:
!python trainer_full_train.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-06-13 10:18:43.253645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749809923.280253    1821 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749809923.287894    1821 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-13 10:18:52.698189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-13 10:18:52.711314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:17498

## 加载模型并推理

In [66]:
eval_model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                                num_labels=len(tags),
                                                                id2label=id2lbl,
                                                                label2id=lbl2id)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

eval_model = eval_model.to(device)

state_dict = torch.load('/kaggle/working/model.bin', map_location=device)

eval_model.load_state_dict(state_dict)

eval_model.eval()

def predict_entities(text):

    batch_texts = [list(text)]

    # 预处理输入文本
    input_data = tokenizer(
        batch_texts, 
        truncation=True, 
        add_special_tokens=False, 
        max_length=512,
        is_split_into_words=True, 
        padding='max_length',
        return_tensors="pt"  
    )

    for key in input_data:
        if isinstance(input_data[key], torch.Tensor):
            input_data[key] = input_data[key].to(device)

    # 运行模型推理
    with torch.no_grad():
        outputs = eval_model(**input_data)

    # 获取预测结果
    predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

    # 获取分词后的 token
    tokens = tokenizer.convert_ids_to_tokens(input_data["input_ids"].squeeze().tolist())

    # 后处理输出，提取实体
    entities = []
    current_entity = None
 
    for i, (token, pred) in enumerate(zip(tokens, predictions)):
        label = id2lbl[pred]
        # if label is not 'O':
        #     print(i, label)
 
        if label.startswith("B-"):
            if current_entity is not None:
                entities.append(current_entity)
            current_entity = {"entity": label[2:], "content": token}
        elif label.startswith("I-") and current_entity is not None and label[2:] == current_entity["entity"]:
            current_entity["content"] += token
        else:
            if current_entity is not None:
                entities.append(current_entity)
                current_entity = None
 
    # 添加最后一个实体（如果有）
    if current_entity is not None:
        entities.append(current_entity)

    for entity in entities:
        entity["entity"] = entity["entity"].lower()

    return entities

# 示例调用
text = "我最近迷上了《星际穿越》这部电影，它的导演是克里斯托弗·诺兰，真的是一位才华横溢的导演。\
    周末我打算约上几个朋友，一起去位于上海市徐汇区的腾讯大厦附近的电竞馆，玩几局《英雄联盟》。\
    听说腾讯最近在深圳总部举办了一场盛大的游戏开发者大会，吸引了全球众多游戏爱好者的关注。\
    另外，我还在读一本关于人工智能的书籍，书名是《人工智能简史》，作者是尼克·波斯特洛姆，\
    他是未来生命研究所的创始人之一，这个组织一直致力于研究人工智能对人类社会的影响。"
entities = predict_entities(text)

for entity in entities:
    print(entity)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'entity': 'position', 'content': '《星际穿越》'}
{'entity': 'organization', 'content': '导演'}
{'entity': 'movie', 'content': '克里斯托弗·诺兰'}
{'entity': 'organization', 'content': '导演'}
{'entity': 'name', 'content': '上海市徐汇区'}
{'entity': 'name', 'content': '腾讯大厦'}
{'entity': 'book', 'content': '《英雄联盟》'}
{'entity': 'scene', 'content': '腾讯'}
{'entity': 'name', 'content': '深圳'}
{'entity': 'government', 'content': '《人工智能简史》'}
{'entity': 'organization', 'content': '作者'}
{'entity': 'movie', 'content': '尼克·波斯特洛姆'}
{'entity': 'organization', 'content': '创始人'}
